In [1]:
import cv2
import numpy as np
import os
from math import tan, atan2, cos, sin, sqrt, pi
from collections import Counter
import pandas as pd
import re

In [2]:
def drawAxis(img, p_, q_, color, scale): 
  p = list(p_) 
  q = list(q_) 

  ## [visualization1] 
  angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians 
  hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0])) 
  # Here we lengthen the arrow by a factor of scale 
  q[0] = p[0] - scale * hypotenuse * cos(angle) 
  q[1] = p[1] - scale * hypotenuse * sin(angle) 
  cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA) 
  # create the arrow hooks 
  p[0] = q[0] + 9 * cos(angle + pi / 4) 
  p[1] = q[1] + 9 * sin(angle + pi / 4) 
  cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA) 
  p[0] = q[0] + 9 * cos(angle - pi / 4) 
  p[1] = q[1] + 9 * sin(angle - pi / 4) 
  cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA) 
  ## [visualization1] 

def getOrientation(pts, img, img_o): 
  ## [pca] 
  # Construct a buffer used by the pca analysis 
  sz = len(pts) 
  data_pts = np.empty((sz, 2), dtype=np.float64) 
  for i in range(data_pts.shape[0]): 
    data_pts[i,0] = pts[i,0,0] 
    data_pts[i,1] = pts[i,0,1]

  # Perform PCA analysis 
  mean = np.empty((0)) 
  mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean) 
  # Store the center of the object 
  cntr = (int(mean[0,0]), int(mean[0,1])) 
  ## [pca] 
  ## [visualization] 
  # Draw the principal components 
  cv2.circle(img, cntr, 3, (255, 0, 255), 2) 
  p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0]) 
  p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0]) 
  drawAxis(img_o, cntr, p1, (255, 255, 0), 1) 
  drawAxis(img_o, cntr, p2, (0, 0, 255), 5) 
  angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians 
  ## [visualization] 
  # Label with the rotation angle 
  label = "  Rotation Angle: " + str(int(np.rad2deg(angle)) + 90) + " degrees" 
  textbox = cv2.rectangle(img_o, (cntr[0], cntr[1]-25), (cntr[0] + 250, cntr[1] + 10), (255,255,255), -1) 
  cv2.putText(img_o, label, (cntr[0], cntr[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA) 
  return angle 

In [ ]:
# Load the image mask and normal image
img = cv2.imread('ternausnet/train_data/masks_with_axons/mask_16.jpg')
img_o = cv2.imread('ternausnet/train_data/images/image_16.tif') 

th, threshed = cv2.threshold(img, 240, 256, cv2.THRESH_BINARY_INV)
#cv2.imshow('Gray Image', threshed) 
#contours = cv2.getStructuringElement(shape=cv2.MORPH_ELLIPSE, ksize=(5, 5))
#closing = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, contours, iterations=1)
threshed = cv2.cvtColor(threshed, cv2.COLOR_BGR2GRAY)
contours, _ = cv2.findContours(threshed, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
for i,c in enumerate(contours):
  area = cv2.contourArea(c)
  # ignore contour which is too small or large
  if area < 1e2 or 1e5 < area:
    continue
  # draw each contour only for visualization
  cv2.drawContours(img_o, contours, i, (0, 0, 255), 2)
  # find orientation of each shape
  angle = getOrientation(c,threshed, img_o)
cv2.imshow('Image with Orientations',img_o)
cv2.waitKey(0) 
cv2.destroyAllWindows() 